In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import warnings

warnings.filterwarnings('ignore')
os.chdir('/content/gdrive/My Drive/RF/')
os.getcwd()

'/content/gdrive/My Drive/RF'

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection, tree, ensemble, metrics

In [ ]:
main_df = pd.read_csv('raw_data/train.csv')

In [ ]:
main_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
del main_df['Name']

In [ ]:
def cabin_split(cabin, first=True):
    if first:
        return cabin.split('/')[0]
    return cabin.split('/')[-1]

main_df['Cabin_F'] = main_df['Cabin'].apply(lambda x: cabin_split(str(x)))
main_df['Cabin_L'] = main_df['Cabin'].apply(lambda x: cabin_split(str(x), first=False))

del main_df['Cabin']

In [ ]:
for i in ['CryoSleep', 'VIP', 'Transported']:
    main_df[i] = main_df[i].astype(bool).astype(int)

In [ ]:
main_df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_F,Cabin_L
0,0001_01,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,P
1,0002_01,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,S
2,0003_01,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,S
3,0003_02,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,S
4,0004_01,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,S


In [ ]:
for i in ['Destination', 'HomePlanet', 'Cabin_F', 'Cabin_L']:
    main_df = pd.concat([main_df,
                         pd.get_dummies(main_df[i], prefix=str(i))],
                        axis=1)
    del main_df[i]

In [ ]:
main_df = main_df.rename(columns={'Destination_55 Cancri e':'D1',
                                  'Destination_PSO J318.5-22':'D2',
                                  'Destination_TRAPPIST-1e':'D3'})

In [ ]:
uid = ['PassengerId']
features = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
            'ShoppingMall', 'Spa', 'VRDeck', 'D1', 'D2', 'D3',
            'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
            'Cabin_F_A', 'Cabin_F_B', 'Cabin_F_C', 'Cabin_F_D',
            'Cabin_F_E', 'Cabin_F_F', 'Cabin_F_G', 'Cabin_F_T', 'Cabin_F_nan',
            'Cabin_L_P', 'Cabin_L_S', 'Cabin_L_nan']
target = ['Transported']

In [ ]:
main_df = main_df[uid+features+target]

In [ ]:
main_df.shape

(8693, 28)

In [ ]:
main_df = main_df.fillna(0)

In [ ]:
main_df.shape

(8693, 28)

In [ ]:
main_df.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,D1,...,Cabin_F_D,Cabin_F_E,Cabin_F_F,Cabin_F_G,Cabin_F_T,Cabin_F_nan,Cabin_L_P,Cabin_L_S,Cabin_L_nan,Transported
0,0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,1,0,0,0
1,0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,0,...,0,0,1,0,0,0,0,1,0,1
2,0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,...,0,0,0,0,0,0,0,1,0,0
3,0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,...,0,0,0,0,0,0,0,1,0,0
4,0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,0,...,0,0,1,0,0,0,0,1,0,1


In [ ]:
train_df, test_df1 = model_selection.train_test_split(main_df, test_size=0.25)

In [ ]:
rf_clf = ensemble.RandomForestClassifier(n_estimators=100,
                                         max_depth=7,
                                         n_jobs=-1)
rf_clf = rf_clf.fit(train_df[features], train_df[target])

In [ ]:
ab_clf = ensemble.AdaBoostClassifier(
            base_estimator=tree.DecisionTreeClassifier(max_depth=1),
            n_estimators=300)
ab_clf = ab_clf.fit(train_df[features], train_df[target])

In [ ]:
train_df['Prediction'] = rf_clf.predict(train_df[features])
test_df1['Prediction'] = rf_clf.predict(test_df1[features])
print(metrics.classification_report(train_df['Transported'], train_df['Prediction']))
print(metrics.classification_report(test_df1['Transported'], test_df1['Prediction']))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81      3236
           1       0.82      0.81      0.82      3283

    accuracy                           0.81      6519
   macro avg       0.81      0.81      0.81      6519
weighted avg       0.81      0.81      0.81      6519

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      1079
           1       0.81      0.79      0.80      1095

    accuracy                           0.80      2174
   macro avg       0.80      0.80      0.80      2174
weighted avg       0.80      0.80      0.80      2174



In [ ]:
train_df['Prediction'] = ab_clf.predict(train_df[features])
test_df1['Prediction'] = ab_clf.predict(test_df1[features])
print(metrics.classification_report(train_df['Transported'], train_df['Prediction']))
print(metrics.classification_report(test_df1['Transported'], test_df1['Prediction']))

              precision    recall  f1-score   support

           0       0.82      0.76      0.79      3236
           1       0.78      0.83      0.81      3283

    accuracy                           0.80      6519
   macro avg       0.80      0.80      0.80      6519
weighted avg       0.80      0.80      0.80      6519

              precision    recall  f1-score   support

           0       0.82      0.76      0.79      1079
           1       0.78      0.83      0.81      1095

    accuracy                           0.80      2174
   macro avg       0.80      0.80      0.80      2174
weighted avg       0.80      0.80      0.80      2174



In [ ]:
test_df = pd.read_csv('raw_data/test.csv')

In [ ]:
del test_df['Name']

In [ ]:
test_df['Cabin_F'] = test_df['Cabin'].apply(lambda x: cabin_split(str(x)))
test_df['Cabin_L'] = test_df['Cabin'].apply(lambda x: cabin_split(str(x), first=False))

del test_df['Cabin']

In [ ]:
for i in ['CryoSleep', 'VIP']:
    test_df[i] = test_df[i].astype(bool).astype(int)

In [ ]:
for i in ['Destination', 'HomePlanet', 'Cabin_F', 'Cabin_L']:
    test_df = pd.concat([test_df,
                         pd.get_dummies(test_df[i], prefix=str(i))],
                        axis=1)
    del test_df[i]

In [ ]:
test_df = test_df.rename(columns={'Destination_55 Cancri e':'D1',
                                  'Destination_PSO J318.5-22':'D2',
                                  'Destination_TRAPPIST-1e':'D3'})

In [ ]:
test_df = test_df[uid+features]

In [ ]:
test_df = test_df.fillna(0)

In [ ]:
test_df['Prediction'] = rf_clf.predict(test_df[features])

In [ ]:
submission_df = test_df[uid+['Prediction']]

In [ ]:
submission_df['Prediction'] = submission_df['Prediction'].astype(bool)
submission_df = submission_df.rename(columns={'Prediction':'Transported'})

In [ ]:
submission_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [ ]:
submission_df.to_csv('202207311630_rf_025_n300_d2.csv',
                     encoding='utf-8', index=False)

In [ ]:
# Names
# Null value imputation
# Gridsearch CV
# 0.25 split
# 